Задача: от каких самых малоприбыльных рейсов из Анапы мы можем отказаться в зимнее время 2017 года.
    
    Модель оценки будем строить по формуле:  стоимость билетов - стоимость топлива на рейс
        
Для решения данной задачи определяемся что зимнее время 2017 это месяцы январь, февраль и декабрь

Скриптом определяем код аэропорта города Анапа:
    
SELECT airport_code
FROM dst_project.airports ap
WHERE ap.city = 'Anapa'

получаем значение AAQ

Получаем общее количество рейсов за интресуемые месяцы

SELECT *
FROM dst_project.flights
WHERE departure_airport = 'AAQ'
  AND (date_trunc('month', scheduled_departure) in ('2017-01-01','2017-02-01', '2017-12-01'))
  AND status not in ('Cancelled')
  
их получается 127

Проанализировав полученный результат видим что в данные месяцы из анапы есть 3 направления: Москва, Белгород, Новокузнецк

Скриптом склеиваем таблицы flights, ticket_flights по ключу flight_id

SELECT *
FROM dst_project.flights f
FULL OUTER JOIN dst_project.ticket_flights tf ON f.flight_id = tf.flight_id
WHERE departure_airport = 'AAQ'
  AND (date_trunc('month', scheduled_departure) in ('2017-01-01',
                                                    '2017-02-01',
                                                    '2017-12-01'))
  AND status not in ('Cancelled')

в результате получаем что для направления Новокузнецк в базе данных нет билетов.
Ну что ж данное направление отбрасываем за неимением данных.

Скриптом 

SELECT *
FROM dst_project.flights
WHERE departure_airport = 'AAQ'
  AND (date_trunc('month', scheduled_departure) in ('2017-12-01'))
  AND status not in ('Cancelled')
  
выясняем что за декабрь 2017 года данные тоже отсутствуют.
Нас это тоже не смущает будем анализировать только два месяца.

После предварительного знакомства с базой данных авиаперелетов готовим итоговый датасет.

WITH a AS
  (SELECT *,
          date_part('hour', f.actual_arrival - f.actual_departure)*60 + date_part('minute', f.actual_arrival - f.actual_departure) time_flight
   FROM dst_project.flights f
   WHERE actual_arrival IS NOT NULL
     AND actual_departure IS NOT NULL ),
     b AS
  (SELECT DISTINCT tf.flight_id flightid,
                   sum(tf.amount) ticket_selling
   FROM dst_project.ticket_flights tf
   GROUP BY flightid)
SELECT *
FROM a aa
JOIN b bb ON aa.flight_id = bb.flightid
WHERE departure_airport = 'AAQ'
  AND (date_trunc('month', scheduled_departure) in ('2017-01-01',
                                                    '2017-02-01',
                                                    '2017-12-01'))
  AND status not in ('Cancelled')
ORDER BY 9

Полученный результат сохраняем в файл dataset-itog.xlsx
Почему был выбран excel? Просто пока изучал SQL подзабыл PYTHON :) Да и в excel можно поизучать данные побогаче чем в csv.

In [ ]:
Блин после запуска ячейки markdown происходит ее переформатирование и она становится хуже читаемой :( 
Почему так происходит я не знаю.

In [16]:
import pandas as pd
from datetime import datetime
anapa = pd.read_excel('dataset-itog.xlsx', index_col=None)
anapa.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   flight_id            118 non-null    int64 
 1   flight_no            118 non-null    object
 2   scheduled_departure  118 non-null    object
 3   scheduled_arrival    118 non-null    object
 4   departure_airport    118 non-null    object
 5   arrival_airport      118 non-null    object
 6   status               118 non-null    object
 7   aircraft_code        118 non-null    object
 8   actual_departure     118 non-null    object
 9   actual_arrival       118 non-null    object
 10  time_flight          118 non-null    int64 
 11  flightid             118 non-null    int64 
 12  ticket_selling       118 non-null    int64 
dtypes: int64(4), object(9)
memory usage: 12.1+ KB


C:\Users\rellik\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Тут на меня напал варнинг. Предположил что в файле сформированным Metabase отсутствует стиль по умолчанию и двинулся дальше.

Удаляем ненужные столбцы - flightid это дубликат flight_id

In [2]:
anapa = anapa.drop(['flightid'], axis=1)

In [3]:
anapa

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival,time_flight,ticket_selling
0,136661,PG0480,2017-01-01T09:25:00Z,2017-01-01T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-01-01T09:27:00Z,2017-01-01T10:17:00Z,50,759000
1,136420,PG0252,2017-01-01T10:05:00Z,2017-01-01T11:45:00Z,AAQ,SVO,Arrived,733,2017-01-01T10:09:00Z,2017-01-01T11:47:00Z,98,1542000
2,136936,PG0480,2017-01-02T09:25:00Z,2017-01-02T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-01-02T09:28:00Z,2017-01-02T10:18:00Z,50,632400
3,136458,PG0252,2017-01-02T10:05:00Z,2017-01-02T11:45:00Z,AAQ,SVO,Arrived,733,2017-01-02T10:07:00Z,2017-01-02T11:46:00Z,99,1568800
4,136937,PG0480,2017-01-03T09:25:00Z,2017-01-03T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-01-03T09:29:00Z,2017-01-03T10:19:00Z,50,626100
...,...,...,...,...,...,...,...,...,...,...,...,...
113,136351,PG0252,2017-02-26T10:05:00Z,2017-02-26T11:45:00Z,AAQ,SVO,Arrived,733,2017-02-26T10:13:00Z,2017-02-26T11:53:00Z,100,1555400
114,136953,PG0480,2017-02-27T09:25:00Z,2017-02-27T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-02-27T09:27:00Z,2017-02-27T10:17:00Z,50,765300
115,136266,PG0252,2017-02-27T10:05:00Z,2017-02-27T11:45:00Z,AAQ,SVO,Arrived,733,2017-02-27T10:08:00Z,2017-02-27T11:48:00Z,100,1566400
116,136844,PG0480,2017-02-28T09:25:00Z,2017-02-28T10:15:00Z,AAQ,EGO,Arrived,SU9,2017-02-28T09:26:00Z,2017-02-28T10:16:00Z,50,575100


Визуально смотрим что загрузилось, а также что нет пропусков (Ура).

В интернете найдены следующие показатели расхода топлива:
SU9 - 1700 кг/час
733 - 2400 кг/час
Так как у нас в итоговом датасете полет в минутах то расход делим на 60 чтобы привести все в единую систему единиц измерения.

In [4]:
consumption_su9 = 1700/60
consumption_733 = 2400/60
print(consumption_su9)
print(consumption_733)

28.333333333333332
40.0


Так же в интернете найдена стоимость авиационного топлива за тонну для города Анапа:
01.2017 - 41435 рублей без ндс
02.2017 - 39553 рублей без ндс
12.2017 - 47101 рублей без ндс
Добавим в наш датасет стоимость потраченного топлива на полет рейса по формуле:
полет в минутах * расход топлива в кг/мин * стоимсоть авиационного топлива за кг

Информацию по какой схеме налогооблажения работал поставщик топлива с аэропортом не нашел, поэтому предположил что работали по упрощенной, т.е. без ндс.

Цену топлива тоже переводим в килограммы.

In [5]:
at_01 = 41435/1000
at_02 = 39553/1000
at_12 = 47101/1000
print(at_01,' ',at_02,' ',at_12)

41.435   39.553   47.101


In [6]:
anapa.actual_departure = anapa['actual_departure'].str[0:-4]

In [7]:
anapa.actual_departure = pd.to_datetime(anapa.actual_departure, format="%Y-%m-%dT%H:%M")


In [8]:
anapa['actual_d_month'] = anapa.actual_departure.dt.month

Добавляем столбец который содержит номер месяца. В нем мы проставим стоимость авиационного топлива в зависимости от месяца.

In [9]:
anapa.actual_d_month = anapa.actual_d_month.apply(lambda x: at_01 if x==1 else at_02)
anapa['aircraft_consumption'] = anapa.aircraft_code.apply(lambda x: consumption_su9 if x=='SU9' else consumption_733)

Создаем столбец расхода топлива для двух типов самолетов (других у нас нет).

In [11]:
anapa['ammount_cost'] = round(anapa.time_flight * anapa.actual_d_month * anapa.aircraft_consumption,0)

In [ ]:
Вычисляем затраты на топливо как произведение времени полета на расход топлива и на стоимость.

anapa['total_profit'] = anapa.ticket_selling - anapa.ammount_cost
anapa['costs/sales'] = round(anapa.total_profit/anapa.ticket_selling*100,2)

In [ ]:
Считаем разницу между продажей билетов и затратами на топливо
Также считаем отношение затрат на топливо к продажам билетов в процентах.

In [14]:
anapa.describe()

,flight_id,time_flight,ticket_selling,actual_d_month,aircraft_consumption,ammount_cost,total_profit,costs/sales
count,118.000000,118.000000,1.180000e+02,118.000000,118.000000,118.000000,1.180000e+02,118.000000
mean,136533.440678,74.957627,1.176506e+06,40.541847,34.166667,109709.949153,1.066796e+06,90.954237
std,261.971994,24.992783,4.794251e+05,0.943790,5.858209,52484.339974,4.284458e+05,1.220610
min,136119.000000,49.000000,5.310000e+05,39.553000,28.333333,54913.000000,4.734740e+05,88.140000
25%,136286.250000,50.000000,7.464000e+05,39.553000,28.333333,57526.000000,6.903670e+05,90.132500
50%,136528.500000,74.500000,1.081650e+06,41.435000,34.166667,107461.000000,9.707635e+05,90.900000
75%,136768.500000,100.000000,1.625850e+06,41.435000,40.000000,162425.000000,1.467357e+06,92.195000
max,136956.000000,103.000000,1.886000e+06,41.435000,40.000000,170712.000000,1.720260e+06,92.680000


In [ ]:
Выводим и изучаем основные статистические характеристики. На их основе делаем вывод.

In [ ]:
Вывод: Модель предложенная в начале задачи является недостаточной для определения прибыльности рейсов. 
Так как затраты на топливо составляют в среднем 9% от суммы проданных билетов. 
Что можно сделать для улучшения оценки? Нужна дополнительная информация для учета всех затрат на полет. 
Такие как зп сотрудников, амортизация оборудования, затраты на обслуживания летного поля и аэропорта,
техобслуживание самолета и т.д.